### 현재 파일 디렉토리 경로

In [1]:
import os

In [2]:
# 현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())

C:\Users\sypar\Desktop\git\py_technique


## 1. 물류 최적경로

In [10]:
# 물류의 기초가 되는 운송최적화를 검토하고 기초적인 기술을 학습
# 실제 물류 데이터로부터 네트워크 구조를 가시화하는 방법을 배우고, 최적 물류 계획을 세우는 흐름을 학습

In [11]:
# 최근 회사 이익이 감소 -> 물류비용을 줄이고 효율화를 생각함
# 제품의 부품을 보관하는 창고에서 생상 공장까지 운송 비용을 낮출 수 있을지 검토 요청

### 데이터 불러오기 및 결합

In [12]:
# tbl_factory.csv : 생산 공장 데이터
# tbl_warehouse.csv : 창고 데이터
# rel_cost.csv : 창고와 공장 간의 운송 비용
# tbl_transaction.csv : 2019년의 공장으로의 부품 운송 실적

In [13]:
import pandas as pd

In [16]:
# 공장 데이터
factories = pd.read_csv("pyda100-master/6장/tbl_factory.csv", index_col = 0)
factories

,FCName,FCDemand,FCRegion
FCID,,,
FC00001,서울공장,28,북부
FC00002,수원공장,29,북부
FC00003,인천공장,31,북부
FC00004,대전공장,25,북부
FC00005,대구공장,21,남부
FC00006,광주공장,30,남부
FC00007,부산공장,25,남부
FC00008,울산공장,16,남부


In [17]:
# 창고 데이터
warehouse = pd.read_csv("pyda100-master/6장/tbl_warehouse.csv", index_col = 0)
warehouse

,WHName,WHSupply,WHRegion
WHID,,,
WH00001,판교창고,35,북부
WH00002,용인창고,41,북부
WH00003,수원창고,42,북부
WH00004,대전창고,60,남부
WH00005,대구창고,72,남부
WH00006,광주창고,65,남부


In [20]:
# 비용 테이블
cost = pd.read_csv('pyda100-master/6장/rel_cost.csv', index_col = 0)
cost.head()

,FCID,WHID,Cost
RCostID,,,
1,FC00001,WH00001,0.4
2,FC00001,WH00002,0.8
3,FC00001,WH00003,1.5
4,FC00002,WH00001,0.9
5,FC00002,WH00002,0.8


In [21]:
# 운송 실적 테이블
trans = pd.read_csv('pyda100-master/6장/tbl_transaction.csv', index_col = 0)
trans.head()

,TransactionDate,ToFC,FromWH,Quantity
TRID,,,,
0,2019-01-01 02:11:10,FC00004,WH00003,33
1,2019-01-01 06:12:42,FC00007,WH00006,19
2,2019-01-01 06:32:32,FC00006,WH00004,31
3,2019-01-01 07:17:06,FC00002,WH00003,18
4,2019-01-01 07:52:18,FC00001,WH00002,30


In [28]:
# 기초 데이터는 운송 실적임
# 운송실적 테이블에 각 테이블 조인(left join)
join_data = pd.merge(trans, cost, left_on = ["ToFC", "FromWH"], right_on = ["FCID", "WHID"], how = 'left')
join_data.head()

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8


In [29]:
# 공장정보 추가
join_data = pd.merge(join_data, factories, left_on= "ToFC", right_on="FCID", how = "left")
join_data.head()

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost,FCName,FCDemand,FCRegion
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1,대전공장,25,북부
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3,부산공장,25,남부
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9,광주공장,30,남부
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6,수원공장,29,북부
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8,서울공장,28,북부


In [30]:
# 창고 정보 추가
join_data = pd.merge(join_data, warehouse, left_on= "FromWH", right_on= "WHID", how ="left")
join_data.head()

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost,FCName,FCDemand,FCRegion,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1,대전공장,25,북부,수원창고,42,북부
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3,부산공장,25,남부,광주창고,65,남부
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9,광주공장,30,남부,대전창고,60,남부
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6,수원공장,29,북부,수원창고,42,북부
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8,서울공장,28,북부,용인창고,41,북부


In [31]:
# 컬럼 정리
join_data = join_data[["TransactionDate", "Quantity", "Cost", "ToFC", "FCName" , "FCDemand", "FromWH", "WHName", "WHSupply", "WHRegion"]]
join_data.head()

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부


In [32]:
# 북부 데이터 추출
north = join_data.loc[join_data["WHRegion"] == "북부"]
north.head()

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부
7,2019-01-01 09:09:30,12,1.5,FC00001,서울공장,28,WH00003,수원창고,42,북부
8,2019-01-01 10:52:55,27,1.5,FC00003,인천공장,31,WH00003,수원창고,42,북부


In [33]:
# 남부 데이터 추출
south = join_data.loc[join_data["WHRegion"] == "남부"]
south.head()

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
5,2019-01-01 08:56:09,31,0.3,FC00005,대구공장,21,WH00005,대구창고,72,남부
6,2019-01-01 09:00:15,33,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부
9,2019-01-01 14:12:51,21,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부


### EDA

In [37]:
# 지사의 비용합계 
# 운송 실적 총비용 집계 결과 : 북부지사 쪽이 남부지사보다 운송비용의 총액이 많이 소요된다.
print("북부지사 총 비용: " + str(north["Cost"].sum()) + "만원")
print("남부지사 총 비용: " + str(south["Cost"].sum()) + "만원")

북부지사 총 비용: 2189.3만원
남부지사 총 비용: 2062.0만원


In [38]:
# 지사의 총 운송개수 
# 운송 실적 총 운송 부품 개수 집계 결과 : 북부지사보다 남부지사 쪽이 많은 부품을 운송하고 있다.
print("북부지사 총부품 운송개수: " + str(north["Quantity"].sum()) + "개")
print("남부지사 총부품 운송개수: " + str(south["Quantity"].sum()) + "개")

북부지사 총부품 운송개수: 49146개
남부지사 총부품 운송개수: 50214개


In [ ]:
# 운송 부품 1개당 운송 비용 : 남부지사가 낮음
